In [33]:
#important imports
import mediapipe as mp
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
from os import listdir
import shutil
from PIL import Image, ImageOps
from collections import deque
from collections import Counter
from tensorflow.keras.models import load_model

model = load_model('fine_tuned_model.h5')

#LEFT = 0 / RIGHT = 1

#setting important variables
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils
threshold = 0.95

def save_landmark (landmarks, label=-1):
    lm = []
    for i in range(33):
        if i not in range(13,23):
            lm.append(landmarks[i].x)
            lm.append(landmarks[i].y)
            lm.append(landmarks[i].z)
            lm.append(landmarks[i].visibility)    
    if(label != -1): lm.append(label)
    return lm

def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
        
    return angle 

In [34]:
from scipy.special import softmax

val_data = pd.read_csv("lunge_landmark.csv",header=None)
predict_val = []
predict_val = val_data.iloc[:,1:-1].values

print(predict_val.shape)         
print(np.array([predict_val[0]]).shape)  
input_data = np.expand_dims(predict_val, axis=0)  # Ensure the correct dimensionality

# Make predictions
test_res0 = model.predict(input_data)
print(softmax(test_res0, axis=1))
print(np.squeeze(test_res0))
print(softmax(test_res0, axis=1))
print(np.squeeze(test_res0))
# print(model.predict_proba(np.array([predict_val[0]])))
print(test_res0[0][0])
print(test_res0[0][1])
# print(test_res0[1])
print(np.argmax(np.squeeze(test_res0)))

test_res1 = model.predict(np.array([predict_val]))
print(softmax(test_res1, axis=1))
print(np.squeeze(test_res1))
# print(model.predict_proba(np.array([predict_val[1]])))
print(np.argmax(np.squeeze(test_res1)))

(2, 92)
(1, 92)
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 749ms/step
[[[0.72555083 0.2744492 ]
  [0.27444917 0.72555083]]]
[[0.9943452  0.00565487]
 [0.0221802  0.97781974]]
[[[0.72555083 0.2744492 ]
  [0.27444917 0.72555083]]]
[[0.9943452  0.00565487]
 [0.0221802  0.97781974]]
[0.9943452  0.00565487]
[0.0221802  0.97781974]
0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 206ms/step
[[[0.72555083 0.2744492 ]
  [0.27444917 0.72555083]]]
[[0.9943452  0.00565487]
 [0.0221802  0.97781974]]
0


In [35]:
prediction = deque(maxlen=6)
prediction.append(2)
prediction.append(2)
prediction.append(2)
prediction.append(2)
prediction.append(2)
prediction.append(2)

cap = cv2.VideoCapture(0)
## Setup mediapipe instance
with pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks
        if results.pose_landmarks:
            landmarks = results.pose_landmarks.landmark
            data = save_landmark(landmarks)
            data = np.array(data, dtype='float32')
            data = np.expand_dims(data, axis=0)
            data = np.expand_dims(data, axis=1) 
            # Save relevant key joints for angle calculations
            # l_SHOULDER = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            # r_SHOULDER = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            l_HIP = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            r_HIP = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
            l_KNEE = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
            r_KNEE = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
            l_ANKLE = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
            r_ANKLE = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]
            # # print(data.shape)
            print(res.shape)
            res = model.predict(data)
            final_res = np.argmax(np.squeeze(res))
            

            if res[0][0][np.argmax(res[0][0])] < threshold:
                # prediction.append(str(2))
                print_res = 2
            else:
                # prediction.append(str(final_res))
                print_res = np.argmax(res[0][0])
                
            prediction.append(print_res)
            # print("Prediction addition:" + str(print_res))
            # print(prediction)
            
        else: res_text = 'Pose Not Detected'    
            # print("Res:", res[0][np.argmax(res)])
        print_res = Counter(prediction).most_common(1)[0][0]
        # print("Most common:" + str(print_res))

            
            # print(data)
            # # Get coordinates
            # shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            # elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            # wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            
            # # Calculate angle
            # angle = calculate_angle(shoulder, elbow, wrist)
            
            # # Visualize angle
            # cv2.putText(image, str(angle), 
            #                tuple(np.multiply(elbow, [640, 480]).astype(int)), 
            #                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
            #                     )
                       
    
        if print_res == 2: res_text = 'Pose Not Detected'
        else: res_text = 'Lunge'
        
        print_text = ' '
        try:
            if print_res != 2: 
                l_leg = calculate_angle(l_HIP, l_KNEE, l_ANKLE)
                r_leg = calculate_angle(r_HIP, r_KNEE, r_ANKLE)
                # l_waist = calculate_angle(l_SHOULDER, l_HIP, l_KNEE)
                # r_waist = calculate_angle(r_SHOULDER, r_HIP, r_KNEE)
                cv2.putText(image, str(l_leg), 
                           tuple(np.multiply(l_KNEE, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
                cv2.putText(image, str(r_leg), 
                           tuple(np.multiply(r_KNEE, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
                if print_res == 1:
                    if l_leg < 70 or l_leg > 110: print_text = 'Square Left Knee'
                    if r_leg < 120: print_text = 'Stretch Right Leg'
                    else: print_text = 'Correct Posture'
                    
                if print_res == 0:
                    if r_leg < 70 or r_leg > 110: print_text = 'Square Right Knee'
                    if l_leg < 120: print_text = 'Stretch Left Leg'
                    else: print_text = 'Correct Posture'
                    
        except: pass
        
        # print(prediction)
        cv2.rectangle(image, (0,0), (640,40), (245,117,16), -1)
        # cv2.putText(image, str(final_res) + res_text + print_text, (3,30), 
        #             cv2.FONT_HERSHEY_SIMPLEX, 1,
        #             (255, 255, 255), 2, cv2.LINE_AA)
        cv2.putText(image, res_text + ' ' + print_text, (3,30), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1,
                    (255, 255, 255), 2, cv2.LINE_AA)
        
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )               
        
        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()



(1, 1, 2)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 124ms/step

c:\Users\HP\AppData\Local\Programs\Python\Python311\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 126ms/step
(1, 1, 2)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
(1, 1, 2)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
(1, 1, 2)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
(1, 1, 2)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
(1, 1, 2)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
(1, 1, 2)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
(1, 1, 2)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
(1, 1, 2)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
(1, 1, 2)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
(1, 1, 2)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
(1, 1, 2)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
(1, 1, 2)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
(1, 1, 2)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
(1, 1, 2)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
(1, 1, 2)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
(1, 1, 2)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
(1, 1, 2)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
(1, 1, 2)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
(1, 1, 2)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
(1, 1, 2)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
(

In [36]:
val_data = pd.read_csv("lunge_landmark.csv",header=None)
predict_val = []
predict_val = val_data.iloc[:,1:-1].values


predict_val = np.array(predict_val)
print(predict_val)

res = model.predict(predict_val)
print (np.argmax(res))

[[ 0.34592879  0.32745191 -0.00137423  0.99994683  0.35409832  0.31430691
  -0.0217468   0.99995708  0.35550424  0.31444573 -0.02176742  0.99995041
   0.35691568  0.31459197 -0.0217938   0.99997139  0.35387051  0.31367755
   0.01579606  0.99995553  0.35514283  0.31335637  0.01585071  0.99994731
   0.35644278  0.31293461  0.01591424  0.99995661  0.37019742  0.32391652
  -0.08088129  0.99996793  0.36930031  0.32123825  0.09025352  0.99984097
   0.35083517  0.34264371 -0.02279375  0.99991882  0.35002196  0.34214863
   0.02638373  0.9998005   0.38755843  0.40842646 -0.16615535  0.99997628
   0.38819054  0.39890391  0.18014731  0.99820292  0.41594759  0.60155874
  -0.09926461  0.9998672   0.39692578  0.59816897  0.09917521  0.99966896
   0.52362865  0.69475889 -0.20143154  0.99939549  0.29253498  0.61128211
   0.07909784  0.98536521  0.6360321   0.77145058 -0.20594129  0.9994185
   0.29931056  0.77774131  0.12643771  0.98942357  0.6601873   0.77480465
  -0.20636328  0.99358529  0.31418335  

ValueError: Exception encountered when calling Sequential.call().

[1mCannot take the length of shape with unknown rank.[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=<unknown>, dtype=float32)
  • training=False
  • mask=None

In [ ]:
# if self.is_body_detected is False:
#             return NONE_ACTION_IDX

#         # res = self.model.predict(np.expand_dims(inp_seq, axis=0), verbose=0)[0]
#         res = self.model.predict(np.expand_dims(inp_seq, axis=0))[0]

#         '''
#         # Since our model has a softmax layer at the end, the output will be
#         # an array of percentages which totals to 1:
#         # EX: [0.1 0.3 0.6]
#         # When we use res[np.argmax(res)], np.argmax(res) will take the index
#         # of the highest element in the list. With res[np.argmax(res)] we get
#         # the highest element.
#         Reason why we don't use max(res) is because we still need the index
#         to get our action string in actions[].
#         But yeah the INDEX from np.argmax(res) is basically our index. 
#         '''
#         # current_action = ""
#         if res[np.argmax(res)] > self.pred_threshold:
#             current_action_idx = np.argmax(res)
#         else:
#             current_action_idx = NONE_ACTION_IDX

#         return current_action_idx